**Importing libraries and dependencies**


In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

**Device configuration**

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"using the device: {device}")

using the device: cuda


**Data ccollection and preprocessing**

In [3]:
data = load_breast_cancer()
x = data.data
y = data.target

**Split the training and test sets**

In [6]:
X_train , X_test ,y_train, y_test = train_test_split(x,y,test_size=0.2,random_state=42)

**Standarize the data using StandardScaler**

In [7]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


**Convert numpy array to tensor and move it to gpu**

In [8]:
X_train = torch.tensor(X_train,dtype=torch.float32).to(device)
y_train = torch.tensor(y_train,dtype=torch.float32).to(device)
X_test = torch.tensor(X_test,dtype=torch.float32).to(device)
y_test = torch.tensor(y_test,dtype=torch.float32).to(device)

**Neural Network Architecture**

In [9]:
class NeuralNetwork(nn.Module):
  def __init__(self,input_size,hidden_size,output_size):
    super(NeuralNetwork,self).__init__()
    self.fc1 = nn.Linear(input_size,hidden_size)
    self.relu = nn.ReLU()
    self.fc2 = nn.Linear(hidden_size,output_size)
    self.sigmoid = nn.Sigmoid()

  def forward(self,x):
    out = self.fc1(x)
    out = self.relu(out)
    out = self.fc2(out)
    out = self.sigmoid(out)
    return out



**Define hyperparameters**

In [14]:
input_size = X_train.shape[1]
hidden_size = 128
output_size = 1
learning_rate = 0.001
num_epochs = 100

**initialize NN and move it to the gpu**

In [15]:
model = NeuralNetwork(input_size,hidden_size,output_size).to(device)

**define loss and optimizer**

In [16]:
critertion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(),lr=learning_rate)

**Training the model**

In [29]:
for epoch in range(num_epochs):
  model.train()
  optimizer.zero_grad()
  outputs = model(X_train)
  loss = critertion(outputs,y_train.reshape(-1,1))
  loss.backward()
  optimizer.step()

  with torch.no_grad():
    predicted = outputs.round()
    correct = (predicted == y_train.reshape(-1,1)).float().sum()
    accuracy = correct/y_train.shape[0]
    if (epoch+1)%10 == 0:
      print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f} Accuracy: %{accuracy.item()*100:.2f}")


Epoch [10/100], Loss: 0.4617 Accuracy: %90.99
Epoch [20/100], Loss: 0.3258 Accuracy: %93.19
Epoch [30/100], Loss: 0.2412 Accuracy: %94.29
Epoch [40/100], Loss: 0.1882 Accuracy: %94.73
Epoch [50/100], Loss: 0.1540 Accuracy: %95.60
Epoch [60/100], Loss: 0.1306 Accuracy: %96.48
Epoch [70/100], Loss: 0.1139 Accuracy: %97.58
Epoch [80/100], Loss: 0.1014 Accuracy: %98.02
Epoch [90/100], Loss: 0.0918 Accuracy: %98.24
Epoch [100/100], Loss: 0.0841 Accuracy: %98.46


**Evaluate against test-set**

In [30]:
model.eval()
with torch.no_grad():
  outputs = model(X_test)
  predicted = outputs.round()
  correct = (predicted == y_test.reshape(-1,1)).float().sum()
  accuracy = correct/y_test.shape[0]
  print(f"Test Accuracy: %{accuracy.item()*100:.2f}")


Test Accuracy: %98.25


**The model scored %98.25 accuracy!**